In [46]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import json
import xml

In [47]:
import pymongo

In [48]:
import xmltodict
from tqdm import tqdm

In [49]:
from datetime import datetime, date

In [50]:
folder = 'D:/Fichiers_dev/test_xml/'

In [51]:
tree = ET.parse(folder + 'declarations.xml')

In [52]:
root = tree.getroot()

In [53]:
declarations_json = []

for declaration in tqdm(root) :
    # pour chaque declaration (1er niveau du parsage), on reconvertie en xml string decodé en utf-8
    declaration_parse = ET.tostring(declaration).decode('utf-8')
    # on convertie en json chaque declaration
    declaration_json = xmltodict.parse(declaration_parse)
    # on enregistre chaque json dans une liste
    declarations_json.append((declaration_json['declaration']))

100%|███████████████████████████████████████████████████████████████████████████| 10893/10893 [00:31<00:00, 342.59it/s]


In [54]:
def remplacer_espace_insecable(dictionnaire, cle, valeur):
    
    # si la valeur est une chaine, faire le remplacement
    dictionnaire[cle] = valeur.replace('\xa0', '')

In [55]:
def conversion(dictionnaire, cle, valeur):
    
    # conversion en numerique
    if (cle=='annee' or cle=='montant'): 
        dictionnaire[cle] = int(valeur)
    
    # conversion en datetime
    if cle=='dateDepot':
        format_date = '%d/%m/%Y %H:%M:%S'
        dictionnaire[cle] = datetime.strptime(valeur, format_date)
    
    # conversion en date
    if cle=='dateNaissance':
        format_date = '%d/%m/%Y'
        # on recupere seulement la date de l'objet datetime
        dictionnaire[cle] = datetime.strptime(valeur, format_date)

In [56]:
def transformation_dictionnaire_recursive_fonction(dictionnaire, fonction):
    for cle, valeur in dictionnaire.items():

        # si la valeur est un dictionnaire, appel récursif
        if isinstance(valeur, dict):
                valeur_dict = valeur
                transformation_dictionnaire_recursive_fonction(dictionnaire=valeur_dict, fonction=fonction)

        # si la valeur est une liste, on boucle sur la liste, puis appel récusrsif
        if isinstance(valeur, list):
                for dictionnaire_in in valeur :
                    transformation_dictionnaire_recursive_fonction(dictionnaire=dictionnaire_in, fonction=fonction)

        if isinstance(valeur, str) :
            fonction(dictionnaire, cle, valeur)

        else : 
            dictionnaire[cle] = valeur

In [72]:
# on applique les fonctions de transformation à toutes les déclarations

for declaration in tqdm(declarations_json):
    
    try : 
    # remplacer les espaces insécables
        transformation_dictionnaire_recursive_fonction(dictionnaire=declaration, fonction=remplacer_espace_insecable)
    except Exception as error:
        print(error)
    finally : continue
    
    try : 
    # faire les conversions numériques
        transformation_dictionnaire_recursive_fonction(dictionnaire=declaration, fonction=conversion)
    except Exception as error:
        print(error)
    finally : continue

100%|██████████████████████████████████████████████████████████████████████████| 10893/10893 [00:01<00:00, 6197.61it/s]


In [73]:
declarations_json[150]

{'dateDepot': datetime.datetime(2022, 9, 14, 11, 27, 8),
 'uuid': 'c2b15a9e-b774-4a71-9d44-cc7d6066b272',
 'origine': 'ADEL',
 'complete': 'true',
 'attachedFiles': {'attachedFiles': {'fileName': 'VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML',
   'serverFileName': None,
   'base64EncodedContent': None}},
 'declarationVersion': '20171221',
 'activConsultantDto': {'items': {'items': [{'motif': {'id': 'CREATION',
      'label': None},
     'commentaire': None,
     'nomEmployeur': 'Autoentreprise',
     'description': 'Conseil aux entreprises',
     'dateDebut': '01/2018',
     'dateFin': '03/2020',
     'remuneration': {'brutNet': 'Brut',
      'montant': {'montant': [{'annee': 2018, 'montant': 0},
        {'annee': 2019, 'montant': 50000},
        {'annee': 2020, 'montant': 30000}]}}},
    {'motif': {'id': 'CREATION', 'label': None},
     'commentaire': None,
     'nomEmployeur': 'Autoentreprise',
     'description': 'Conseil aux entreprise',
     'dateDebut': '03/2014',
     'dateFin': '02/2017',

### Connection à MongoDB

In [74]:
client = pymongo.MongoClient('mongodb://localhost:27017')

In [75]:
db = client['declarations']

collection = db['declarations_all']

### Conversion en json et transfert dans MongoDB

In [76]:
collection.drop()

for declaration in tqdm(declarations_json):
    collection.insert_one(declaration)

100%|██████████████████████████████████████████████████████████████████████████| 10893/10893 [00:05<00:00, 1967.50it/s]


### Supression des déclarations autres que la derniere declaration pour chaque individus

In [77]:
folder = 'D:/Data_Analyst_Projet/Autres cours/NoSQL/'

In [78]:
liste_uuid = list(pd.read_csv(folder+'uuid_derniere_declaration.csv')['uuid'].values)
liste_uuid[:10]

['fa8d18ec-0db9-4a39-b1f4-caba0c31329b',
 'c40ab214-b21e-43b6-95a2-36d5860dc526',
 'ac693815-1521-4a79-a314-6bd830de8988',
 '61853e75-80cc-457b-865c-80456d4a0985',
 'a2893249-f253-4cb4-bb9d-f7fcec066d31',
 '2b6da666-15d3-4864-9231-7a7dd5ae9dda',
 '71a9d498-0801-40f2-bab2-b44126698b19',
 '900e1c2b-91a1-487a-9d03-fbe1ae6831bf',
 '3e209d45-8656-45ac-a8c8-451542abe786',
 'b0268727-af55-4a2a-98fb-6bf6b88cd82b']

In [79]:
result = collection.delete_many({'uuid': {'$nin': liste_uuid}})

In [80]:
print(f"{result.deleted_count} documents supprimés")

5006 documents supprimés


### Requete

In [83]:
print(f"{collection.count_documents({})} documents dans la base de données")

5881 documents dans la base de données


In [82]:
collection.find_one()

{'_id': ObjectId('6591921d964fef9c4bd47d67'),
 'dateDepot': datetime.datetime(2022, 11, 27, 18, 18, 23),
 'uuid': 'fa8d18ec-0db9-4a39-b1f4-caba0c31329b',
 'origine': 'ADEL',
 'complete': 'true',
 'attachedFiles': {'attachedFiles': {'fileName': 'VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML',
   'serverFileName': None,
   'base64EncodedContent': None}},
 'declarationVersion': '20171221',
 'activConsultantDto': {'neant': 'true'},
 'activProfCinqDerniereDto': {'items': {'items': {'motif': {'id': 'CREATION',
     'label': None},
    'commentaire': 'Fonction exercée du 20 Mai au 4 Juillet 2022 \n        [Données non publiées]',
    'description': "Ministre des Solidarités, de l'Autonomie et des Personnes handicapées",
    'employeur': "MINISTERE DES SOLIDARITES, DE L'AUTONOMIE ET DES PERSONNES HANDICAPEES",
    'remuneration': {'brutNet': 'Net',
     'montant': {'montant': {'annee': 2022, 'montant': 10312}}},
    'dateDebut': '05/2022',
    'dateFin': '07/2022'}},
  'neant': 'false'},
 'activProfConjoi